In [ ]:
# Automatically reload custom code modules when there are changes:
%load_ext autoreload
%autoreload 2


In [ ]:
# Adjust relative path so that the notebook can find the code modules:
import sys
sys.path.append('code/dqn/')
sys.path.append('code/')


In [ ]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib notebook


In [ ]:
# Import code modules:
from structures import RingRoad
from animations import Animation
from learning import Game
from learning import Replay


In [ ]:
# Hide warnings about safe distance violation (still in development):
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


# Train:


In [ ]:
# Define a ring road environment:
road_params = {
    'num_avs' : 2,
    'av_even_spacing' : True,
    'num_vehicles' : 10,  # The vechicles at index 0 is an A.V.
    'ring_length' : 100.0,  # The road is a cicle.
    'starting_noise' : 1.0,  # Uniformly add noise to starting positions.
    'temporal_res' : 0.5,  # Set the size of simulation steps (seconds).
    'av_activate' : 30,  # Set when the PID controller is activated.
    'seed' : 286,  # Set a random seed.
}
road = RingRoad(**road_params)

# Plot initial conditions:
#fig,ax = road.plot_ring(step=0, draw_cars_to_scale=True, draw_safety_buffer=True)

# Run the simulation for set number of time steps:
total_time = 90  # In seconds.
total_steps = int(np.ceil(total_time/road.dt))
road.run(steps=total_steps)

# Show step of simulation:
step = road.step
fig, axs = road.plot_dashboard(
    step=step, total_steps=total_steps,
    draw_cars_to_scale=True, draw_safety_buffer=True, label_step=True, label_cars=True,
)


In [ ]:
# Build animation:
anim = Animation(road, speedup=5.0, interval=5)
anim.animate_dashboard(draw_cars_to_scale=True, draw_safety_buffer=False, show_sigma=True)

# Show animation:
anim.show()

# Save animation as GIF (may take a few minutes -- pauses preview above):
anim.save_gif(filepath="outputs/learning_goal.gif", overwrite=True)


In [ ]:
# Deactivate interactive plotting mode (ends preview above):
# anim.stop()


In [ ]:
# Define the same ring road but with learning mode:
road_params['learning_mode'] = True
road_params['av_activate'] = 0
road = RingRoad(**road_params)

# Initialize a game with this road:
game = Game(
    road = road,
    agent_commands = [-1.0, -0.1, 0.0, 0.1, 1.0],
    past_steps = 3,
    max_seconds = None,
)


In [ ]:
print("observation_space shape:",game.observation_space.shape)
print("action_space size:",game.action_space.n)


In [ ]:
print(game.action_space())


In [ ]:
# Get command tuple at given index:
game.action_space.decode(action_index=3)


In [ ]:
# Get index of command tuple:
game.action_space.encode(agent_commands=(-1,0))


In [ ]:
# Take a few steps:
next_observation, reward, done, info = game.step(3)
next_observation, reward, done, info = game.step(0)
next_observation


In [ ]:
display(game.road.get_vehicle_pos_table().tail())
display(game.road.get_vehicle_control_table().tail())


# Test:


In [ ]:
replay = Replay.from_files(
    saved_models_dir = './code/dqn/saved-models/',
    # run_id = '2n769rgx', ep_num = 4468,
    # run_id = '3al8vr8k', ep_num = 1388,
    # run_id = '2gts27uw', ep_num = 7355,
    run_id = 'fnwz2w4x', ep_num = 5098,
)
replay.build(seconds=30)
env = replay.road


In [ ]:
# Build animation:
anim = Animation(env)
anim.animate_dashboard(draw_cars_to_scale=True, draw_safety_buffer=False, show_sigma=True)


In [ ]:
# Save animation as GIF (may take a few minutes -- pauses preview above):
anim.save_gif(filepath="outputs/learning_result.gif", overwrite=True)
